In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fd-dataset/FD.docx


In [2]:
pip install boto3 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.1 requires botocore<1.35.24,>=1.35.16, but you have botocore 1.29.165 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers datasets faiss-cpu boto3 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 46.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you h

In [5]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain_together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 15.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import numpy as np
import faiss
import pandas as pd
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

# Initialize your LLM model
chat_model = ChatTogether(
    together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

# Prompt engineering: Make the context more specific and improve the instructions
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer (be specific and avoid hallucinations):"
    )
)

# Load the embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Example dummy text (can be replaced with actual data)
dummy_text = ["""
Bajaj customers: Interest rates range from 7.16% to 8.40% per annum (p.a.)
- Senior citizens: Interest rates range from 7.39% to 8.65% p.a., with an additional rate benefit of up to 0.40% p.a.
- Non-Resident Indians (NRIs): Interest rates of up to 8.35% p.a.
- Systematic Deposit Plan (SDP): A regular monthly deposit scheme
Bajaj Finance FDs are considered to be safe and secure, with an AAA(Stable) rating from ICRA and AAA/STABLE from CRISIL.
The minimum investment amount is Rs. 15,000, and the tenure can range from 12 to 60 months.
Before booking an FD, it's important to check the eligibility criteria and document requirements on the lender's website.
"""]

# Create embeddings
embeddings = embedding_model.embed_documents(dummy_text)
embeddings = np.array(embeddings)
dimension = embeddings.shape[1]

# Initialize FAISS index and add embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Setup the docstore and mapping between index and documents
docstore = InMemoryDocstore({0: Document(page_content=dummy_text[0])})
index_to_docstore_id = {0: 0}

# Initialize the FAISS vector store
vector_store = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query  # Use the embedding function
)

# Define the QA chain for answering questions
qa_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template
)

# Example usage
if __name__ == "__main__":
    # Define context with relevant information
    context = """Bajaj Finance offers a variety of fixed deposit (FD) options with different interest rates and terms:
    Regular customers: Interest rates range from 7.16% to 8.40% per annum (p.a.)
    Senior citizens: Interest rates range from 7.39% to 8.65% p.a., with an additional rate benefit of up to 0.40% p.a.
    Non-Resident Indians (NRIs): Interest rates of up to 8.35% p.a.
    Systematic Deposit Plan (SDP): A regular monthly deposit scheme
    Bajaj Finance FDs are considered to be safe and secure, with an AAA(Stable) rating from ICRA and AAA/STABLE from CRISIL.
    The minimum investment amount is Rs. 15,000, and the tenure can range from 12 to 60 months."""

    # Define the question
    question = "What is the interest rate for senior citizens for FD?"

    # Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question,clean_up_tokenization_spaces=False)

    # Print the result
    print(answer)

/tmp/ipykernel_17/3408683879.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_17/3408683879.py:65: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(
/tmp/ipykernel_17/3408683879.py:85: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(context=context, question=question,clean_up_tokenization_spaces=False)


For senior citizens, the interest rate for Bajaj Finance FD ranges from 7.39% to 8.65% p.a., with an additional rate benefit of up to 0.40% p.a.


In [10]:
!pip install pandas python-docx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.9 MB/s eta 0:00:00


In [11]:
# Step 1: List files in the dataset directory
import os

dataset_dir = '/kaggle/input/fd-dataset/'
files = os.listdir(dataset_dir)
for file in files:
    print(file)

# Step 2: Read the Word file if it exists
from docx import Document

# Path to the Word file in the Kaggle dataset
word_file_path = '/kaggle/input/fd-dataset/FD.docx'

# Read the Word file
doc = Document(word_file_path)

# Extract text from the document
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Join all paragraphs into a single string
document_text = '\n'.join(full_text)
print(document_text)


FD.docx
Fixed Deposits Data Collection

FD Interest Rates | Check Latest Fixed Deposit Interest Rates 2024

FD Interest Rates - Check Latest Fixed Deposit Rates 2024 Check the latest FD interest rates for General Citizens up to 8.40% p.a. and for Senior Citizens (60+ years age) up to 8.65% p.a.

Bajaj Finance offers attractive FD rates up to 8.65% p.a. for senior citizens and up to 8.40% p.a. for customers below 60 years of age. With an end-to-end online process, now earn profitable returns, and enjoy the benefits of flexible payout. Start investing with just Rs. 15,000 and earn higher returns for a tenure of 12 to 60 months. Bajaj Finance FD is the highest [ICRA]AAA(Stable) and CRISIL AAA/STABLE rated fixed deposit offering high FD interest rates and the security that the investor's money is entirely safe and remains unaffected by market conditions.

Fixed Deposit (FD) rates for customers below the age of 60 at Bajaj Finance Digital FD (as of 17 Sep, 2024) are as follows:
- Maximum de

In [12]:
import numpy as np
import faiss
import pandas as pd
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

# Initialize your LLM model
chat_model = ChatTogether(
    together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

# Prompt engineering: Make the context more specific and improve the instructions
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer (be specific and avoid hallucinations):"
    )
)

# Load the embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Example dummy text (can be replaced with actual data)
dummy_text =[document_text]# 
'''
["""
Bajaj customers: Interest rates range from 7.16% to 8.40% per annum (p.a.)
- Senior citizens: Interest rates range from 7.39% to 8.65% p.a., with an additional rate benefit of up to 0.40% p.a.
- Non-Resident Indians (NRIs): Interest rates of up to 8.35% p.a.
- Systematic Deposit Plan (SDP): A regular monthly deposit scheme
Bajaj Finance FDs are considered to be safe and secure, with an AAA(Stable) rating from ICRA and AAA/STABLE from CRISIL.
The minimum investment amount is Rs. 15,000, and the tenure can range from 12 to 60 months.
Before booking an FD, it's important to check the eligibility criteria and document requirements on the lender's website.
"""]
'''
# Create embeddings
embeddings = embedding_model.embed_documents(dummy_text)
embeddings = np.array(embeddings)
dimension = embeddings.shape[1]

# Initialize FAISS index and add embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Setup the docstore and mapping between index and documents
docstore = InMemoryDocstore({0: Document(page_content=dummy_text[0])})
index_to_docstore_id = {0: 0}

# Initialize the FAISS vector store
vector_store = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query  # Use the embedding function
)

# Define the QA chain for answering questions
qa_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template
)

# Example usage
if __name__ == "__main__":
    # Define the question
    question = "What is SDP"#"What is the interest rate for senior citizens for FD?"

    # Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)
    
    # Search the vector store for the most similar documents
    D, I = index.search(np.array([query_embedding]), k=1)
    
    # Retrieve the most relevant document
    doc_id = I[0][0]
    document = docstore.search(doc_id)
    context = document.page_content
    
    # Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)

    # Print the result
    print(answer)


A Systematic Deposit Plan (SDP) is a type of investment plan offered by Bajaj Finance, where an individual can invest a fixed amount of money at regular intervals (usually monthly) for a specific tenure. The interest rate is pre-determined by the bank and varies based on the tenure and amount invested. The SDP offers the option to earn interest upon maturity or regular payouts.


In [13]:
document.page_content==document_text

True